In [ ]:
#############################################################
#                     Naver Webtton Crwaling                #
#############################################################

import requests
import csv
from bs4 import BeautifulSoup
import re
import json
import time
import argparse
import requests
import pathlib2
import openpyxl
from urllib.parse import urlparse, parse_qs
import pandas as pd

#import pymysql
#import mysql.connector
#import cryptography

#naver_comment_db = mysql.connector.connect(
#    user='root', 
#    password='alsdk0306', 
#    port='3306',
#    host='127.0.0.1', 
#   database='naver_comment_db', 
#    charset='utf8',
#    auth_plugin='mysql_native_password'
#)

#cursor = naver_comment_db.cursor(dictionary=True)

colHeader=['제목','에피소드화','아이디','댓글내용']
df=pd.DataFrame(columns=colHeader)

def get_commentList(title, titleId, episode_no, comment_page):
    ajax_link = 'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon' + \
                '&pool=cbox3&_callback=jQuery112409768039369695578_1588773317723&lang=ko&country=KR&objectId=' + \
                titleId + '_' + episode_no + '&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=default&page=' + str(comment_page) + \
                '&refresh=true&sort=new&cleanbotGrade=2&_=1588773317725'
    custom_header = {
        'referer': 'https://comic.naver.com/comment/comment.nhn?titleId=' + titleId + '&no=' + episode_no,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
    }
    req = requests.get(ajax_link, headers=custom_header).text
    json_req = json.loads(req[req.find("success") - 2:len(req) - 2])
    commentList = json_req['result']['commentList']
    return commentList

def save_comments(title, titleId, episode_no):
    
    comment_page = 1
   
    cnt=0
    num=0

    while True:
        cnt+=1
        commentList = get_commentList(title, titleId, episode_no, comment_page)
        if commentList:
            for comment in commentList:
              num+=1;
              all_comments_info= {}
              all_comments_info['제목']=title
              all_comments_info['에피소드화']=episode_no
              all_comments_info['아이디']=comment['maskedUserId']
              all_comments_info['댓글내용']=comment['contents'].replace("\n","")
              print(all_comments_info)
              df.loc[len(df)]=all_comments_info
    
            #  sql = """INSERT INTO naver_webtoon_comment (title, user_id, comment) VALUES( '""" + title + """', '""" + comment['maskedUserId'] + """', '""" + comment['contents'].replace("\n","") +"""')""" 
            #  print(sql) 
            #  cursor.execute(sql)
        else:
            return
        comment_page = comment_page + 1



url_episode = 'https://comic.naver.com/webtoon/genre.nhn'
url_omnibus = 'https://comic.naver.com/webtoon/genre.nhn?genre=omnibus'
url_story = 'https://comic.naver.com/webtoon/genre.nhn?genre=story'

url_list = [url_episode, url_omnibus, url_story]
title_list = []
for i in url_list:
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    temp = soup.find_all('dl')
    for j in temp:
        title_list.append(j)

cnt=0
for a in title_list:
    cnt+=1
    print(cnt, "번째 웹툰 크롤링 중...")
    webtoon_info = [cnt]
    webtoon_info.append(a.find('a').get('title'))
    print("제목 : ", a.find('a').get('title'))
    webtoon_info.append(a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
    print("작가 : ", a.find('dd').get_text()[1:len(a.find('dd').get_text())-1])
   
    webtoon_url =  "https://comic.naver.com" + a.find('a').get('href')
    response = requests.get(webtoon_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(webtoon_url)

    webtoon_round_url=soup.find('td',class_='title').find('a').get('href')
    print(soup.find('td',class_='title').find('a').get('href'))

    #id, 회차정보, 요일
    tId=webtoon_round_url.split('titleId=')[1].split('&')[0]
    tRound=webtoon_round_url.split('no=')[1].split('&')[0]
    tWeekday=webtoon_round_url.split('weekday=')[1].split('&')[0]
    webtoon_info.append(tId)
    webtoon_info.append(tRound)
    webtoon_info.append(tWeekday)

    print("아이디: ",tId)
    print("회차정보: ", tRound)
    print("요일: ",tWeekday)

    num=0;
    for i in reversed(range(int(tRound)+1)):
      num+=1
      # https://comic.naver.com/webtoon/detail.nhn?titleId=733766&no=56&weekday=mon 형식
      webtoon_comment_url="https://comic.naver.com/webtoon/detail.nhn?"+ "titleId="+tId+"&no="+str(i)+"&weekday="+tWeekday
      save_comments(a.find('a').get_text(),tId,str(i))
      if(num==10):
        break
      
#naver_comment_db.commit()
#naver_comment_db.close()
df.to_parquet('naver_webtoon_comment.parquet', compression='gzip')
print(cnt, "개 웹툰 크롤링 종료")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'제목': '인생존망', '에피소드화': '54', '아이디': 's010****', '댓글내용': '심슨이라는 친구만있어도 반은성공한거임 ㄹㅇ'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'rang****', '댓글내용': '짐승으로 알아챈거야?ㅋㅋㅋ'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'hugh****', '댓글내용': '전부 성공한 미래여서 좋은데 진우하고 슬기 너무 궁금하다'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'wlca****', '댓글내용': '음 상상도 못한 엔딩이군 ㄹㅇ'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'ejle****', '댓글내용': '작가님 시즌2하신다고 저번화 작가의 말에서 언급하셨습니다'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'sean****', '댓글내용': '아....왜 장안철이 행복해졌지...'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'jjes****', '댓글내용': '나만 마지막에 눈물 났냐'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'taeh****', '댓글내용': '오우어우 선욱이형,태준이형 막화가 2개라니 선넘었는걸?'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'dand****', '댓글내용': '원래 엔딩..연령대..ㅎ 작가님ㅎㅎ 무슨 엔딩이였길래...ㅎㅎ'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'peo0****', '댓글내용': '뭐야 '}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'whdw****', '댓글내용': '구상민 너무 불쌍한데 그냥 연결해주세요'}
{'제목': '인생존망', '에피소드화': '54', '아이디': 'yj11****', '댓글내용': 

In [ ]:
pip install pathlib2